In [6]:
import osimport rasterio
from rasterio.windows import Window
from pathlib import Path
from glob import glob

# Parameters
input_dir = Path("/home/dika/Deep_learning/Torch/Deep_Learning_Unet_Torch/Unet_stadi_file/f")
output_dir = Path("/home/dika/Deep_learning/Torch/Deep_Learning_Unet_Torch/Unet_stadi_file_sliced")
output_dir.mkdir(parents=True, exist_ok=True)

# Patch size
patch_size = 64

# Get all tif files
tif_files = sorted(input_dir.glob("*.tif"))
total_count = 0

for tif_path in tif_files:
    with rasterio.open(tif_path) as src:
        width, height = src.width, src.height
        filename_stem = tif_path.stem

        count = 0
        for top in range(0, height, patch_size):
            for left in range(0, width, patch_size):
                window = Window(left, top, patch_size, patch_size)
                transform = src.window_transform(window)

                try:
                    data = src.read(window=window)
                    if data.shape[1] != patch_size or data.shape[2] != patch_size:
                        continue
                except:
                    continue

                output_filename = f"{filename_stem}_{count:03d}.tif"
                output_path = output_dir / output_filename

                with rasterio.open(
                    output_path, 'w',
                    driver='GTiff',
                    height=patch_size,
                    width=patch_size,
                    count=src.count,
                    dtype=src.dtypes[0],
                    crs=src.crs,
                    transform=transform
                ) as dst:
                    dst.write(data)

                count += 1
                total_count += 1

print(f"✅ Finished slicing {len(tif_files)} files into {total_count} tiles.")


✅ Finished slicing 0 files into 0 tiles.


In [1]:
import os
from pathlib import Path
import rasterio
from rasterio.windows import Window
from glob import glob

# Paths
image_paths = sorted(glob("/home/dika/Deep_learning/Torch/Deep_Learning_Unet_Torch/Unet_stadi_file/image_*.tif"))
mask_paths = sorted(glob("/home/dika/Deep_learning/Torch/Deep_Learning_Unet_Torch/Unet_stadi_file/mask_*.tif"))

print(f"Found {len(image_paths)} image files.")
print(f"Found {len(mask_paths)} mask files.")

output_image_dir = Path("/home/dika/Deep_learning/Torch/Deep_Learning_Unet_Torch/training/image")
output_mask_dir = Path("/home/dika/Deep_learning/Torch/Deep_Learning_Unet_Torch/training/mask")
output_image_dir.mkdir(parents=True, exist_ok=True)
output_mask_dir.mkdir(parents=True, exist_ok=True)

patch_size = 512
count = 0

for idx, (image_file, mask_file) in enumerate(zip(image_paths, mask_paths)):
    print(f"Processing pair {idx}:\n  Image: {image_file}\n  Mask: {mask_file}")
    with rasterio.open(image_file) as img_src, rasterio.open(mask_file) as msk_src:
        width, height = img_src.width, img_src.height
        print(f"  Size: width={width}, height={height}")

        for top in range(0, height, patch_size):
            for left in range(0, width, patch_size):
                window = Window(left, top, patch_size, patch_size)

                try:
                    img_patch = img_src.read(window=window)
                    msk_patch = msk_src.read(window=window)

                    if img_patch.shape[1] != patch_size or img_patch.shape[2] != patch_size:
                        print("  Skipped incomplete patch")
                        continue

                    img_transform = img_src.window_transform(window)
                    msk_transform = msk_src.window_transform(window)

                    image_name = f"image_{count:03d}.tif"
                    mask_name = f"mask_{count:03d}.tif"

                    with rasterio.open(
                        output_image_dir / image_name, 'w',
                        driver='GTiff',
                        height=patch_size,
                        width=patch_size,
                        count=img_src.count,
                        dtype=img_src.dtypes[0],
                        crs=img_src.crs,
                        transform=img_transform
                    ) as dst_img:
                        dst_img.write(img_patch)

                    with rasterio.open(
                        output_mask_dir / mask_name, 'w',
                        driver='GTiff',
                        height=patch_size,
                        width=patch_size,
                        count=msk_src.count,
                        dtype=msk_src.dtypes[0],
                        crs=msk_src.crs,
                        transform=msk_transform
                    ) as dst_msk:
                        dst_msk.write(msk_patch)

                    print(f"  Saved patch: {image_name}, {mask_name}")
                    count += 1

                except Exception as e:
                    print(f"  Error processing patch at ({left}, {top}): {e}")
                    continue

print(f"✅ Sliced {count} image-mask pairs into 256x256 tiles.")


Found 1 image files.


NameError: name 'mask_paths' is not defined